# Gemeindedaten laden und ElCom-IDs extrahieren

In diesem Abschnitt werden die Gemeindedaten aus einer CSV-Datei geladen und die zugehörigen ElCom-IDs über die VESE-API abgefragt.

In [ ]:
# Bibliotheken importieren
import os
import requests
import csv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Gemeindedaten aus CSV-Datei einlesen
gemeinden = []
with open('../data/vese/Gemeindestand.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Kopfzeile überspringen
    for row in reader:
        gemeinden.append(row[4])  # Spalte 4 enthält die Gemeinde-ID (idofs)

# Lizenzschlüssel für die VESE-API
LICENSE_KEY = 'vkmshstyf3r9epknhajqhenzog4opv6wcajd7ob7'

def abfrage_gemeinde(idofs, license_key):
    """
    Fragt die VESE-API für eine bestimmte Gemeinde-ID ab.
    Nutzt einen Retry-Mechanismus für mehr Robustheit.
    """
    url = f'https://opendata.vese.ch/pvtarif/api/getData/muni?idofs={idofs}&licenseKey={license_key}'
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    try:
        response = session.get(url, timeout=20)
        if response.status_code != 200:
            print(f"Fehler bei der Abfrage für Gemeinde {idofs}: Status {response.status_code}")
            return {'valid': False}
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Abfrage für Gemeinde {idofs}: {e}")
        return {'valid': False}

# Ergebnisse der API-Abfragen sammeln
results = []

# Über alle Gemeinden iterieren und API abfragen
for index, gemeinde in enumerate(gemeinden, start=1):
    if index % 100 == 0 or index == len(gemeinden):
        progress = (index / len(gemeinden)) * 100
        print(f"Verarbeite Gemeinde {index} von {len(gemeinden)} ({progress:.2f}%)")
    result = abfrage_gemeinde(gemeinde, LICENSE_KEY)
    if result.get('valid'):
        for evu in result.get('evus', []):
            results.append([
                evu.get('nrElcom', ''),
                evu.get('Name', ''),
                evu.get('idofs', ''),
                evu.get('PLZ', ''),
                evu.get('Gemeinde', '')
            ])

# Ergebnisse in CSV-Datei speichern
output_file = '../data/vese/gemeinde_ergebnisse.csv'
if os.path.exists(output_file):
    os.remove(output_file)

with open(output_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['nrElcom', 'Name', 'idofs', 'PLZ', 'Gemeinde'])  # Kopfzeile
    writer.writerows(results)

# ElCom-Daten für alle EVUs und Jahre abfragen

In diesem Abschnitt werden für alle gefundenen ElCom-IDs und für die Jahre 2015 bis 2024 die Tarifdaten über die VESE-API geladen.

In [ ]:
# Bibliotheken importieren
import csv
import os
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ElCom-IDs aus der Ergebnisdatei extrahieren (Duplikate vermeiden)
elcoms = set()
with open('../data/vese/gemeinde_ergebnisse.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Kopfzeile überspringen
    for row in reader:
        elcoms.add(row[0])  # ElCom-ID (nrElcom) hinzufügen

elcoms = list(elcoms)
print(f"Anzahl eindeutiger ElCom-IDs: {len(elcoms)}")

# Lizenzschlüssel für die VESE-API
LICENSE_KEY = 'vkmshstyf3r9epknhajqhenzog4opv6wcajd7ob7'

# Abzufragende Jahre (2015 bis 2024)
years = list(range(2015, 2025))

def abfrage_ew(evuId, year, license_key):
    """
    Fragt die VESE-API für eine bestimmte ElCom-ID und ein Jahr ab.
    Nutzt einen Retry-Mechanismus für mehr Robustheit.
    """
    year_short = str(year)[-2:]
    url = f'https://opendata.vese.ch/pvtarif/api/getData/evu?evuId={evuId}&year={year_short}&licenseKey={license_key}'
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    try:
        response = session.get(url, timeout=20)
        if response.status_code != 200:
            print(f"Fehler bei der Abfrage für ElCom {evuId}, Jahr {year}: Status {response.status_code}")
            return {'valid': False}
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Abfrage für ElCom {evuId}, Jahr {year}: {e}")
        return {'valid': False}

# Ergebnisse der API-Abfragen sammeln
results = []

# Für jedes Jahr und jede ElCom-ID die Daten abfragen
for year_index, year in enumerate(years, start=1):
    print(f"Verarbeite Jahr {year_index} von {len(years)}: {year}")
    for elcom_index, elcom in enumerate(elcoms, start=1):
        if elcom_index % 100 == 0 or elcom_index == len(elcoms):
            print(f"  Verarbeite ElCom {elcom_index} von {len(elcoms)}")
        result = abfrage_ew(elcom, year, LICENSE_KEY)
        if result.get('valid'):
            evu_data = {
                'year': year,
                'nrElcom': result.get('nrElcom'),
                'nomEw': result.get('nomEw'),
                'link': result.get('link'),
                'tarif1': result.get('tarif1'),
                'tarif2': result.get('tarif2'),
                'tarif3': result.get('tarif3'),
                'explText': result.get('explText'),
                'counterCost': result.get('counterCost'),
                'loadCurveCost': result.get('loadCurveCost'),
                'nbrPowerCat': result.get('nbrPowerCat'),
                'power1': result.get('power1'),
                'energy1': result.get('energy1'),
                'eco1': result.get('eco1'),
                'energyAuto1': result.get('energyAuto1'),
                'ecoAuto1': result.get('ecoAuto1'),
                'power2': result.get('power2'),
                'energy2': result.get('energy2'),
                'eco2': result.get('eco2'),
                'energyAuto2': result.get('energyAuto2'),
                'ecoAuto2': result.get('ecoAuto2'),
                'power3': result.get('power3'),
                'energy3': result.get('energy3'),
                'eco3': result.get('eco3'),
                'energyAuto3': result.get('energyAuto3'),
                'ecoAuto3': result.get('ecoAuto3'),
                'power4': result.get('power4'),
                'htnt': result.get('htnt'),
                'stwt': result.get('stwt'),
                'ecoIncl': result.get('ecoIncl'),
                'autot': result.get('autot'),
                'energy1_bfe': result.get('energy1_bfe'),
                'energy1_ht': result.get('energy1_ht'),
                'energy1_nt': result.get('energy1_nt'),
                'energy1_wht': result.get('energy1_wht'),
                'energy1_wnt': result.get('energy1_wnt'),
                'energyAuto1_bfe': result.get('energyAuto1_bfe'),
                'energyAuto1_ht': result.get('energyAuto1_ht'),
                'energyAuto1_nt': result.get('energyAuto1_nt'),
                'energyAuto1_wht': result.get('energyAuto1_wht'),
                'energyAuto1_wnt': result.get('energyAuto1_wnt'),
                'energy2_bfe': result.get('energy2_bfe'),
                'energy2_ht': result.get('energy2_ht'),
                'energy2_nt': result.get('energy2_nt'),
                'energy2_wht': result.get('energy2_wht'),
                'energy2_wnt': result.get('energy2_wnt'),
                'energyAuto2_bfe': result.get('energyAuto2_bfe'),
                'energyAuto2_ht': result.get('energyAuto2_ht'),
                'energyAuto2_nt': result.get('energyAuto2_nt'),
                'energyAuto2_wht': result.get('energyAuto2_wht'),
                'energyAuto2_wnt': result.get('energyAuto2_wnt'),
                'energy3_bfe': result.get('energy3_bfe'),
                'energy3_ht': result.get('energy3_ht'),
                'energy3_nt': result.get('energy3_nt'),
                'energy3_wht': result.get('energy3_wht'),
                'energy3_wnt': result.get('energy3_wnt'),
                'energyAuto3_bfe': result.get('energyAuto3_bfe'),
                'energyAuto3_ht': result.get('energyAuto3_ht'),
                'energyAuto3_nt': result.get('energyAuto3_nt'),
                'energyAuto3_wht': result.get('energyAuto3_wht'),
                'energyAuto3_wnt': result.get('energyAuto3_wnt'),
                'ht_mofr_on': result.get('ht_mofr_on'),
                'ht_mofr_off': result.get('ht_mofr_off'),
                'ht_sa_on': result.get('ht_sa_on'),
                'ht_sa_off': result.get('ht_sa_off'),
                'ht_su_on': result.get('ht_su_on'),
                'ht_su_off': result.get('ht_su_off'),
                'neg': result.get('neg'),
                'neg_text': result.get('neg_text'),
                'pos': result.get('pos'),
                'pos_text': result.get('pos_text'),
                'leist': result.get('leist'),
                'leist_power': result.get('leist_power'),
                'leist_tax': result.get('leist_tax')
            }
            results.append(evu_data)

# Ergebnisse in CSV-Datei speichern
output_file = '../data/vese/evu_ergebnisse.csv'
if os.path.exists(output_file):
    os.remove(output_file)

if results:
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(results[0].keys())  # Kopfzeile
        writer.writerows([evu.values() for evu in results])
    print(f"Ergebnisse wurden in {output_file} gespeichert.")
else:
    print("Keine Ergebnisse zum Speichern vorhanden.")